In [ ]:
from torch.utils.data import DataLoader
from warnings import simplefilter
import matplotlib.pyplot as plt
from dataloader import *
from Trainer import *
import pandas as pd
import numpy as np
from ANN_Models import *
import torch

simplefilter(action='ignore', category=pd.errors.ParserWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=FutureWarning)

In [27]:
store_loss = np.genfromtxt("store_loss.csv", delimiter=',')

In [32]:
lowest_loss = 100
for na in range(5,7):
    for nb in range(1,16):
        print(na, nb)
        data = DATA(na, nb) 
        Xtrain, Ytrain = data.Xtrain, data.Ytrain
        Xval, Yval = data.Xval, data.Yval

        # Load data and set them up correctly for dataloader
        Dataset_train, Dataset_test = CustomDataset(Xtrain.to_numpy(), Ytrain.to_numpy()).split_data([0.9, 0.1])
        Dataset_val = CustomDataset(Xval.to_numpy(), Yval.to_numpy())
        #Dataset_test = CustomDataset(data.testsub.u.to_numpy(), data.testsub.th.to_numpy())

        dl_train = DataLoader(Dataset_train, batch_size=32, shuffle=True)
        dl_val = DataLoader(Dataset_val, batch_size=32, shuffle=False)
        dl_test = DataLoader(Dataset_test, batch_size=32, shuffle=True)
        model = NARX(5)

        train_module = Trainer(model, dl_train, dl_val, dl_test, DIR='savefolderpytorch\\NARX')

        loss = train_module.fit(epochs=10, batch_size=32, save_log=False)
        if loss < lowest_loss:
            lowest_loss = loss
            best_na = na
            best_nb = nb
        store_loss[na, nb-1] = loss

5 1
The device that will be used in training is Quadro P1000
Epoch 1


Training:  78%|███████▊  | 1755/2250 [00:08<00:02, 208.40it/s, loss=0.00478]

In [ ]:
plt.imshow(store_loss)
plt.title( "2-D Heat Map" )
plt.show()

In [ ]:
print("best na: ", best_na)
print("best nb: ", best_nb)
print("lowest loss: ", lowest_loss)

In [ ]:
store_loss

In [ ]:
np.savetxt("store_loss.csv", store_loss, delimiter=",")